# Fact Checking Notebook Demo
This notebook enables experimentation with contrastive knowledge assessment. You can run models including `Flan-t5`, `Bert`, and `GPT2` (among many others, see [repo](https://github.com/daniel-furman/Capstone#model-families-tested)).
The goal is to probe if factual statements are predicted at a higher probability than a given counterfactual.

<a target="_blank" href="https://colab.research.google.com/github/daniel-furman/Capstone/blob/main/notebooks/fact_checking_run_main_demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


## Dependencies

In [ ]:
!git clone https://github.com/daniel-furman/Capstone.git
!pip install -r /content/Capstone/requirements.txt

## Imports

In [ ]:
import os
import torch
import json
from datasets import load_dataset

In [ ]:
if not torch.cuda.is_available():
    raise Exception("Change runtime type to include a GPU.")

In [ ]:
os.chdir('/content/Capstone/src/fact_checking_scripts')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

## Notebook usage

In [ ]:
# import the main wrapper function for running cka

from run_from_config import main
config = {}

Here, you can specify a causal, masked, or text2text large language model in ```config["models"]```. The `transformers` package is used to load the model into the session at mixed 8bit precision (`load_in_8_bit=True`). See [README](https://github.com/daniel-furman/Capstone#models-tested) for the full list of model families supported and HuggingFace for specific model tags.  

Some example VRAM usages:
* `google/flan-ul2` requires slightly more than 32GB VRAM
* `EleutherAI/gpt-neox-20b` requires slightly more than 22GB VRAM
* `EleutherAI/gpt-j-6B` requires slightly more than 7GB VRAM
* `google/flan-t5-xl`requires slightly more than 5GB VRAM
* `roberta-large` requires slightly less than 1GB VRAM



In [ ]:
config["models"] = [
    "distilroberta-base",
]

Next, new facts can be input in ```config["input_information"]```.
  * For instance: to input "Lebron James is famous for playing the sport of {true: basketball; false: football}", see the last example. 

In [ ]:
input_info = [
            {"stem": "The 2020 Olympics were held in",
            "true": "Tokyo",
            "false": "['London', 'Berlin', 'Chicago']",},

            {"stem": "Operation Overlord took place in",
            "true": "Normandy",
            "false": "['Manila', 'Santiago', 'Baghdad']",},

            {"stem": "Steve Jobs is the founder of",
            "true": "Apple",
            "false": "['Microsoft', 'Google', 'Facebook']",},

            {"stem": "Lebron James is famous for playing the sport of",
            "true": "basketball",
            "false": "['football']",},
    ]

with open('/content/input_info_json.json', 'w') as f:
    f.write(json.dumps(input_info))

In [ ]:
config["input_information"] = load_dataset("json", 
                                           data_files="/content/input_info_json.json"
                                           )['train']

Lastly, verbosity controls how much info is printed to the user.

In [ ]:
config["verbosity"] = True

Now, we're ready to run the cka pipeline.

In [ ]:
score_dicts = main(config)

In [ ]:
print(score_dicts[0])

In [ ]:
print(score_dicts[1])

## CLI for running the full fact-checking benchmark

In [ ]:
# full benchmark dataset

!python main.py --model gpt2 --language en